In [ ]:
import gdown
from torch.nn import DataParallel


In [ ]:
file_id = '1D1OxhivWv53ay0xrdF4CHuuywMgiFEmq'
output_file = 'output.zip'

In [ ]:
gdown.download(f'https://drive.google.com/uc?id={file_id}', output_file, quiet=False)

In [ ]:
import zipfile

with zipfile.ZipFile('output.zip', 'r') as zip_ref:
    zip_ref.extractall('./data')

In [1]:
import torch
import logging
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from PIL import Image
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader, random_split
import os
from matplotlib import pyplot as plt
from tqdm import tqdm
from torch import optim
import logging
from torch.utils.tensorboard import SummaryWriter

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
logging.basicConfig(format = "%(asctime)s - %(levelname)s: %(message)s", level = logging.INFO, datefmt = "%I:%M:%S")

In [3]:
class DiffusionForImages:
    def __init__(self, sBeta = 1e-4, eBeta=0.02, timesteps=1000, imgsize = 256, device = "cuda"):
        self.sBeta= sBeta
        self.eBeta = eBeta
        self.timesteps = timesteps
        self.imgsize = imgsize
        self.device = device
        
        self.beta = self.noise_scheduler().to(device)
        self.alpha = 1. - self.beta
        self.alpha_hat = torch.cumprod(self.alpha, dim = 0)

    def noise_scheduler(self):
        return torch.linspace(self.sBeta, self.eBeta, self.timesteps)

    def noising_images(self, x, t):
        sqrt_alpha_hat = torch.sqrt(self.alpha_hat[t])[:, None, None, None]
        sqrt_one_minus_alpha_hat = torch.sqrt(1 - self.alpha_hat[t])[:, None, None, None]

        eta = torch.randn_like(x)

        return sqrt_alpha_hat * x + sqrt_one_minus_alpha_hat * eta, eta


    def get_timesteps(self, n):
        return torch.randint(low=1, high = self.timesteps, size=(n,))

    def sample_image(self, model, n):
        logging.info(f"Sampling {n} new images....")
        model.eval()
        with torch.no_grad():
            x = torch.randn((n,3,self.imgsize, self.imgsize)).to(self.device)
            for i in tqdm(reversed(range(1, self.timesteps)), position = 0):
                t = (torch.ones(n)*i).long().to(self.device)
                predicted_noise = model(x,t)
                alpha = self.alpha[t][:, None, None, None]
                alpha_hat = self.alpha_hat[t][:, None,None,None]
                beta = self.beta[t][:, None, None, None]

                if i>1:
                    noise = torch.randn_like(x)

                else:
                    noise = torch.zeros_like(x)

                x = 1/torch.sqrt(alpha) * (x - ((1-alpha)/(torch.sqrt(1-alpha_hat)))*predicted_noise) + torch.sqrt(beta)*noise

        model.train()
        x = (x.clamp(-1,1)+1)/2
        x = (x*255).type(torch.uint8)

        return x



In [4]:
#different modules
class SelfAttention(nn.Module):
    def __init__(self, channels, size):
        super(SelfAttention, self).__init__()
        self.channels = channels
        self.size = size
        self.mha = nn.MultiheadAttention(channels, 4, batch_first=True)
        self.ln = nn.LayerNorm([channels])
        self.ff_self = nn.Sequential(
            nn.LayerNorm([channels]),
            nn.Linear(channels, channels),
            nn.GELU(),
            nn.Linear(channels, channels),
        )

    def forward(self, x):
        x = x.view(-1, self.channels, self.size * self.size).swapaxes(1, 2)
        x_ln = self.ln(x)
        attention_value, _ = self.mha(x_ln, x_ln, x_ln)
        attention_value = attention_value + x
        attention_value = self.ff_self(attention_value) + attention_value
        return attention_value.swapaxes(2, 1).view(-1, self.channels, self.size, self.size)


class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels, mid_channels=None, residual=False):
        super().__init__()
        self.residual = residual
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.GroupNorm(1, mid_channels),
            nn.GELU(),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.GroupNorm(1, out_channels),
        )

    def forward(self, x):
        if self.residual:
            return F.gelu(x + self.double_conv(x))
        else:
            return self.double_conv(x)


class Down(nn.Module):
    def __init__(self, in_channels, out_channels, emb_dim=256):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, in_channels, residual=True),
            DoubleConv(in_channels, out_channels),
        )

        self.emb_layer = nn.Sequential(
            nn.SiLU(),
            nn.Linear(
                emb_dim,
                out_channels
            ),
        )

    def forward(self, x, t):
        x = self.maxpool_conv(x)
        emb = self.emb_layer(t)[:, :, None, None].repeat(1, 1, x.shape[-2], x.shape[-1])
        # print("x= ",x.shape, "t=", t.shape)
        return x + emb


class Up(nn.Module):
    def __init__(self, in_channels, out_channels, emb_dim=256):
        super().__init__()

        self.up = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True)
        self.conv = nn.Sequential(
            DoubleConv(in_channels, in_channels, residual=True),
            DoubleConv(in_channels, out_channels, in_channels // 2),
        )

        self.emb_layer = nn.Sequential(
            nn.SiLU(),
            nn.Linear(
                emb_dim,
                out_channels
            ),
        )

    def forward(self, x, skip_x, t):
        x = self.up(x)
        x = torch.cat([skip_x, x], dim=1)
        x = self.conv(x)
        emb = self.emb_layer(t)[:, :, None, None].repeat(1, 1, x.shape[-2], x.shape[-1])
        return x + emb


class UNet(nn.Module):
    def __init__(self, c_in=3, c_out=3, time_dim=256, device="cuda"):
        super().__init__()
        self.device = device
        self.time_dim = time_dim
        self.inc = DoubleConv(c_in, 64)
        self.down1 = Down(64, 128)
        self.sa1 = SelfAttention(128, 32)
        self.down2 = Down(128, 256)
        self.sa2 = SelfAttention(256, 16)
        self.down3 = Down(256, 256)
        self.sa3 = SelfAttention(256, 8)

        self.bot1 = DoubleConv(256, 512)
        self.bot2 = DoubleConv(512, 512)
        self.bot3 = DoubleConv(512, 256)

        self.up1 = Up(512, 128)
        self.sa4 = SelfAttention(128, 16)
        self.up2 = Up(256, 64)
        self.sa5 = SelfAttention(64, 32)
        self.up3 = Up(128, 64)
        self.sa6 = SelfAttention(64, 64)
        self.outc = nn.Conv2d(64, c_out, kernel_size=1)

    def pos_encoding(self, t, channels):
        inv_freq = 1.0 / (
            10000
            ** (torch.arange(0, channels, 2, device=self.device).float() / channels)
        )
        pos_enc_a = torch.sin(t.repeat(1, channels // 2) * inv_freq)
        pos_enc_b = torch.cos(t.repeat(1, channels // 2) * inv_freq)
        pos_enc = torch.cat([pos_enc_a, pos_enc_b], dim=-1)
        return pos_enc
    

    def forward(self, x, t):
        t = t.unsqueeze(-1).type(torch.float)
        t = self.pos_encoding(t, self.time_dim)

        #encoder (performs downsampling)
        x1 = self.inc(x)
        x2 = self.down1(x1, t)
        x2 = self.sa1(x2)
        x3 = self.down2(x2, t)
        x3 = self.sa2(x3)
        x4 = self.down3(x3, t)
        x4 = self.sa3(x4)

        #bottleneck
        x4 = self.bot1(x4)
        x4 = self.bot2(x4)
        x4 = self.bot3(x4)

        #decoder (performs upsampling)
        x = self.up1(x4, x3, t)
        x = self.sa4(x)
        x = self.up2(x, x2, t)
        x = self.sa5(x)
        x = self.up3(x, x1, t)
        x = self.sa6(x)
        output = self.outc(x)
        return output



In [5]:
import matplotlib.pyplot as plt

def plot_images(images, ncols=4):
    n_images = len(images)
    nrows = (n_images + ncols - 1) // ncols

    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(10, 10))

    for i, image in enumerate(images):
        row = i // ncols
        col = i % ncols

        ax = axes[row, col]
        ax.imshow(image.permute(1, 2, 0).cpu())
        ax.axis('off')

    # Hide any empty subplots
    for i in range(n_images, nrows * ncols):
        row = i // ncols
        col = i % ncols
        axes[row, col].axis('off')

    plt.tight_layout()
    plt.show()



def save_images(images, path, **kwargs):
    grid = torchvision.utils.make_grid(images, **kwargs)
    ndarr = grid.permute(1, 2, 0).to('cpu').numpy()
    im = Image.fromarray(ndarr)
    im.save(path)


def get_data(args):
    # Define your transformations
    transforms = torchvision.transforms.Compose([
        torchvision.transforms.Resize(80),
        torchvision.transforms.RandomResizedCrop(args.image_size, scale=(0.8, 1.0)),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    # Load the entire dataset
    full_dataset = torchvision.datasets.ImageFolder(args.dataset_path, transform=transforms)

    # Choose the number of images you want to use (e.g., 10,000)
    num_images_to_use = 10000

    # Create a random subset of the dataset with the desired number of images
    subset_dataset, _ = random_split(full_dataset, [num_images_to_use, len(full_dataset) - num_images_to_use])

    # Create a dataloader for the subset dataset
    dataloader = DataLoader(subset_dataset, batch_size=args.batch_size, shuffle=True)

    return dataloader

def setup_logging(run_name):
    os.makedirs("models", exist_ok=True)
    os.makedirs("results", exist_ok=True)
    os.makedirs(os.path.join("models", run_name), exist_ok=True)
    os.makedirs(os.path.join("results", run_name), exist_ok=True)

In [10]:
from torch import optim
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
from torch.nn import DataParallel

def train(args):
    setup_logging(args.run_name)
    device = args.device
    model = UNet()
    
    model = DataParallel(model)
#     checkpoint_path = "./models/DDPM/ckpt.pt"
#     model.load_state_dict(torch.load(checkpoint_path))
    model = model.to(device)
    dataloader = get_data(args)

    optimizer = optim.AdamW(model.parameters(), lr=args.lr)
    mse = nn.MSELoss()
    diffusion = DiffusionForImages(imgsize=args.image_size, device=device)
    logger = SummaryWriter(os.path.join("runs", args.run_name))
    l = len(dataloader)
    
    count = 0
    
    for epoch in range(args.epochs):
        count+=1
        logging.info(f"Starting epoch {epoch}:")
        pbar = tqdm(dataloader)
        for i, (images, _) in enumerate(pbar):
            images = images.to(device)
            # print(f"Batch {i} is on GPU {images.device.index}")
            
            t = diffusion.get_timesteps(images.shape[0]).to(device)
#             print("t= ",t)
            x_t, noise = diffusion.noising_images(images, t)
            # print("x_t= ", x_t.shape)
            predicted_noise = model(x_t, t)
            loss = mse(noise, predicted_noise)
            # print("Outside: input size", images.size())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            pbar.set_postfix(MSE=loss.item())
            logger.add_scalar("MSE", loss.item(), global_step=epoch * l + i)

        sampled_images = diffusion.sample_image(model, n=images.shape[0])
#         save_images(sampled_images, os.path.join("results", args.run_name, f"{epoch+276}.jpg"))
        if count%5==0:
            torch.save(model.state_dict(), os.path.join("models", args.run_name, f"ckpt.pt"))
            save_images(sampled_images, os.path.join("results", args.run_name, f"{epoch+276}.jpg"))

In [7]:
def begin():
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument('-f')
    args = parser.parse_args()
    args.run_name = "DDPM"
    args.epochs = 500
    args.batch_size = 17
    args.image_size = 64
    args.dataset_path = "/kaggle/input/casia-webface/casia-webface"
    args.device = torch.device("cuda")
    args.lr = 3e-4
    train(args)

In [8]:
# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [11]:
if __name__ == '__main__':
    begin()

100%|██████████| 589/589 [04:58<00:00,  1.97it/s, MSE=0.0178]
999it [00:50, 19.69it/s]
100%|██████████| 589/589 [04:00<00:00,  2.45it/s, MSE=0.0152]
999it [00:50, 19.72it/s]
100%|██████████| 589/589 [04:07<00:00,  2.38it/s, MSE=0.0145] 
999it [00:52, 18.99it/s]
100%|██████████| 589/589 [04:07<00:00,  2.38it/s, MSE=0.00656]
999it [00:53, 18.82it/s]
100%|██████████| 589/589 [04:03<00:00,  2.41it/s, MSE=0.0135] 
999it [00:50, 19.96it/s]
100%|██████████| 589/589 [04:03<00:00,  2.42it/s, MSE=0.0178] 
999it [00:50, 19.70it/s]
100%|██████████| 589/589 [04:01<00:00,  2.44it/s, MSE=0.0705] 
999it [00:50, 19.97it/s]
100%|██████████| 589/589 [04:01<00:00,  2.44it/s, MSE=0.0885] 
999it [00:50, 19.80it/s]
100%|██████████| 589/589 [04:00<00:00,  2.45it/s, MSE=0.0739] 
999it [00:50, 19.60it/s]
100%|██████████| 589/589 [04:00<00:00,  2.44it/s, MSE=0.0033] 
999it [00:49, 20.01it/s]
100%|██████████| 589/589 [04:02<00:00,  2.43it/s, MSE=0.0109] 
999it [00:50, 19.80it/s]
100%|██████████| 589/589 [04:00<00

KeyboardInterrupt: 

In [ ]:
device = "cuda"

# Load the checkpt 
checkpoint = torch.load('./models/DDPM/ckpt.pt')

# Remove the 'module.' prefix from keys if present (for single GPU)
state_dict = {k.replace('module.', ''): v for k, v in checkpoint.items()}


model = UNet().to(device)  
model.load_state_dict(state_dict)


diffusion = DiffusionForImages(imgsize=64, device=device)
x = diffusion.sample_image(model, n=16)
plot_images(x)

In [ ]:
    device = "cuda"
    model = UNet()
    model = DataParallel(model)
    ckpt = torch.load("./models/DDPM/ckpt.pt")
    model.load_state_dict(ckpt)
    model = model.to(device)
    diffusion = DiffusionForImages(imgsize=64, device=device)
    x = diffusion.sample_image(model, n=64)
    print(x.shape)
    plot_images(x)
    save_images(x, os.path.join("results", "DDPM", "a.jpg"))


In [ ]:
# import torch
# import torchvision
# from torchvision import transforms
# from PIL import Image

# # Load pre-trained WideResNet50 model
# model = torchvision.models.wide_resnet50_2(pretrained=True)
# model.eval()

# # Transformation for resizing images to 224x224
# transform = transforms.Compose([
#     transforms.Resize((256, 256)),
#     transforms.ToTensor(),
# ])

# transform2 = transforms.Compose([
#     transforms.Resize((512, 512ed)),
#     transforms.ToTensor(),
# ])

# # Example RGB image (assuming img is your input image)
# # Replace this with your own image data
# img = Image.open('/kaggle/input/casia-webface/casia-webface/000000/00000001.jpg')


# # Transform the image
# input_image = transform(img).unsqueeze(0)  # Add a batch dimension
# v=input_image.shape
# print(v)

# # Access intermediate layers for feature extraction
# features = list(model.children())[:-3]  # Get layers excluding the last two (avgpool and fc)
# feature_extractor = torch.nn.Sequential(*features)

# # Extract features from the original image
# with torch.no_grad():
#     original_features = feature_extractor(input_image)
#     original_features = original_features.squeeze(dim=0)

# # Resize and split the RGB image into 4 non-overlapping patches
# patches = []  # Store the 4 patches
# patch_size = (224, 224)
# width, height = img.size
# for i in range(2):
#     for j in range(2):
#         left = j * patch_size[0]
#         upper = i * patch_size[1]
#         right = left + patch_size[0]
#         lower = upper + patch_size[1]
#         patch = img.crop((left, upper, right, lower))
#         patches.append(transform(patch).unsqueeze(0))  # Add each patch to the list

# # Extract features for each patch
# patch_features = []
# with torch.no_grad():
#     for patch in patches:
#         patch_feature = feature_extractor(patch)
# #         patch_feature = patch_feature.squeeze(dim=0)
#         patch_features.append(patch_feature)

# Feature maps for the original image (size: 1024 x 14 x 14)
print(f"Feature maps for the original image: {original_features.shape}")

# feature_cat = torch.cat(original_features, dim=0)
# feature_cat.shape



In [ ]:
# import torch
# import torchvision
# from torchvision import transforms
# from PIL import Image

# # Load pre-trained WideResNet50 model
# model = torchvision.models.wide_resnet50_2(pretrained=True)
# model.eval()

# # Transformation for resizing images to 224x224
# transform = transforms.Compose([
#     transforms.Resize((256, 256)),
#     transforms.ToTensor(),
# ])

# transform2 = transforms.Compose([
#     transforms.Resize((512, 512)),
# #     transforms.ToTensor(),
# ])

# tensor = transforms.Compose([
#     transforms.ToTensor(),
# ])


# features = list(model.children())[:-3]  # Get layers excluding the last two (avgpool and fc)
# feature_extractor = torch.nn.Sequential(*features)

# patch_size = (256, 256)


# dataset_path = '/kaggle/input/casia-webface/casia-webface'

# for image_file in os.listdir(dataset_path):
#     image_pth = os.path.join(dataset_path, image_file)

#     for image in os.listdir(image_pth):
#         image_path = os.path.join(image_pth, image)
#         img = Image.open(image_path)
#         width, height = img.size
#         input_image = transform(img).unsqueeze(0)
        
#         with torch.no_grad():
#             f1 = feature_extractor(input_image)
# #             f1 = f1.squeeze(dim=0)
            
#         #scale2 feature extraction
#         img2 = transform2(img)
        
#         patches=[]
#         patch_features=[]
#         for i in range(2):
#             for j in range(2):
#                 left = j * patch_size[0]
#                 upper = i * patch_size[1]
#                 right = left + patch_size[0]
#                 lower = upper + patch_size[1]
#                 patch = img2.crop((left, upper, right, lower))
#                 patches.append(tensor(patch).unsqueeze(0))
        
#         with torch.no_grad():
#             for patch in patches:
#                 patch_feature = feature_extractor(patch)
#                 patch_features.append(patch_feature)
#         patch_features.append(f1)
#         final_feature = torch.cat(patch_features, dim=0)
        
#         #storing the feature tensor
#         output_folder = f'/kaggle/working/{image_file}/'

#         # Check if the folder exists, and create it if not
#         if not os.path.exists(output_folder):
#             os.makedirs(output_folder)
#         file_path = os.path.join(output_folder, f'{image}.pt')

#         # Open the file and save the tensor
#         with open(file_path, 'wb') as file:
#             torch.save(final_feature, file)



In [ ]:
from torch.utils.data import DataLoader
ds = '/kaggle/input/casia-webface'
dataloader = DataLoader(ds, batch_size=32, shuffle=True)

import torch
import torchvision
from torchvision import transforms
from PIL import Image

# Load pre-trained WideResNet50 model
model1 = torchvision.models.wide_resnet50_2(pretrained=True)
model1.eval()

# Transformation for resizing images to 224x224
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

transform2 = transforms.Compose([
    transforms.Resize((512, 512)),
#     transforms.ToTensor(),
])

tensor = transforms.Compose([
    transforms.ToTensor(),
])


features_layer = list(model1.children())[:-3]  # Get layers excluding the last two (avgpool and fc)
feature_extractor = torch.nn.Sequential(*features_layer)

patch_size = (256, 256)



def feature_function(img):
    
    width, height = img.size
    input_image = transform(img).unsqueeze(0)

    with torch.no_grad():
        f1 = feature_extractor(input_image)
#             f1 = f1.squeeze(dim=0)

    #scale2 feature extraction
    img2 = transform2(img)

    patches=[]
    patch_features=[]
    for i in range(2):
        for j in range(2):
            left = j * patch_size[0]
            upper = i * patch_size[1]
            right = left + patch_size[0]
            lower = upper + patch_size[1]
            patch = img2.crop((left, upper, right, lower))
            patches.append(tensor(patch).unsqueeze(0))

    with torch.no_grad():
        for patch in patches:
            patch_feature = feature_extractor(patch)
            patch_features.append(patch_feature)
    patch_features.append(f1)
    final_feature = torch.cat(patch_features, dim=0)
    return final_feature
        
    
def train_branch_2(args):
    setup_logging(args.run_name)
    device = args.device
    model = UNet()
    
    model = DataParallel(model)
    checkpoint_path = "./models/DDPM/ckpt.pt"
    model.load_state_dict(torch.load(checkpoint_path))
    model = model.to(device)
    dataloader = get_data(args)

    optimizer = optim.AdamW(model.parameters(), lr=args.lr)
    mse = nn.MSELoss()
    diffusion = DiffusionForImages(imgsize=args.image_size, device=device)
    logger = SummaryWriter(os.path.join("runs", args.run_name))
    l = len(dataloader)
    
    count = 0
    
    for epoch in range(args.epochs):
        count+=1
        logging.info(f"Starting epoch {epoch}:")
        pbar = tqdm(dataloader)
        for i, (images, _) in enumerate(pbar):
            images = images.to(device)
            # print(f"Batch {i} is on GPU {images.device.index}")
            batch_features = []

            # Perform feature extraction on-the-fly for each image in the batch
            for image in images:
                feature = feature_function(image)
                batch_features.append(feature)

            # Convert the list of features to a tensor
            features = torch.stack(batch_features)
            
            t = diffusion.get_timesteps(features.shape[0]).to(device)
#             print("t= ",t)
            x_t, noise = diffusion.noising_images(features, t)
            # print("x_t= ", x_t.shape)
            predicted_noise = model(x_t, t)
            loss = mse(noise, predicted_noise)
            # print("Outside: input size", images.size())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            pbar.set_postfix(MSE=loss.item())
            logger.add_scalar("MSE", loss.item(), global_step=epoch * l + i)

        sampled_images = diffusion.sample_image(model, n=features.shape[0])
#         save_images(sampled_images, os.path.join("results", args.run_name, f"{epoch+276}.jpg"))
        torch.save(model.state_dict(), os.path.join("models", args.run_name, f"ckpt.pt"))

In [ ]:
def feature_scale_1(model):
    model.eval()

    # Transformation for resizing images to 224x224
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])
    

    

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!pip install keras_retinanet

In [ ]:
from keras_resnet import models
from keras.layers import Input

# Define input tensor shape
input_shape = (800, 800, 3)  # Adjust the shape according to your input images

# Create an input tensor
inputs = Input(shape=input_shape)

# Initialize ResNet2D50 model with the input tensor
model = models.ResNet2D50(inputs)

In [ ]:
import cv2
import os
import numpy as np
from keras_retinanet import models
from keras_retinanet.utils.image import preprocess_image, resize_image

confidence_threshold = 0.5

# Load the pre-trained RetinaNet model
model = models.load_model('/kaggle/input/object-detection-model/resnet50_coco_best_v2.1.0.h5')

# Path to CASIA-WebFace images
dataset_path = '/kaggle/input/casia-webface/casia-webface'

# Iterate through images
for image_file in os.listdir(dataset_path):
    image_pth = os.path.join(dataset_path, image_file)

    for image in os.listdir(image_pth):
        image_path = os.path.join(image_pth, image)
        # Load and preprocess the image
        imge = cv2.imread(image_path)
        imge = preprocess_image(imge)
        imge, scale = resize_image(imge)

        # Use the model to predict faces
        boxes, scores, labels = model.predict(np.expand_dims(imge, axis=0))

        # Filter out low confidence detections
        high_confidence_indices = np.where(scores[0] > confidence_threshold)[0]
        print(high_confidence_indices)
        boxes = boxes[0, high_confidence_indices]
        print("boxes",boxes)
        scores = scores[0, high_confidence_indices]
        print("scores",scores)

        # Extract and save faces
        for box in boxes:
            x, y, x2, y2 = box.astype(int)
            face = imge[y:y2, x:x2]
            print("in")
            output_folder = f'/kaggle/working/{image_file}/'

            # Check if the folder exists, and create it if not
            if not os.path.exists(output_folder):
                os.makedirs(output_folder)
            cv2.imwrite(os.path.join(output_folder, f'{image}.jpg'), face)
            print("out")

        